## Setup
This notebook requires some packages besides pytorch-lightning.

In [1]:
! pip install --quiet "pandas" "torch" "torchvision" "ipython[notebook]" "seaborn" "pytorch-lightning" "torchmetrics" "lightning-bolts"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.8/300.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 54.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Run this if you intend to use TPUs
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

In [4]:
# import sys
# sys.path.insert(0,'/content/drive/MyDrive/S13/S13')

In [5]:
import os
os.chdir("/content/drive/MyDrive/S13/S13")
!ls

FileNotFoundError: ignored

In [ ]:
import os

import pandas as pd
import seaborn as sn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from IPython.core.display import display
#from pl_bolts.datamodules import CIFAR10DataModule
#from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.functional import accuracy

seed_everything(7)

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 32 if torch.cuda.is_available() else 16
NUM_WORKERS = int(os.cpu_count() / 2)

In [ ]:
import config
import torch
import torch.optim as optim
import PASCAL_VOC
from model import YOLOv3
from tqdm import tqdm
from utils import (
    mean_average_precision,
    cells_to_bboxes,
    get_evaluation_bboxes1,
    save_checkpoint,
    load_checkpoint,
    check_class_accuracy,
    get_loaders,
    plot_couple_examples
)
from loss import YoloLoss
from dataset import YOLODataset
import warnings
warnings.filterwarnings("ignore")

### PASCAL_VOC Data Module

Import the existing data module from `bolts` and modify the train and test transforms.

In [ ]:
# import module
import torch

# To get the layers and losses for our model
from torch import nn
import pytorch_lightning as pl

# To get the activation function for our model
import torch.nn.functional as F

# To get MNIST data and transforms
from torchvision import datasets, transforms

# To get the optimizer for our model
from torch.optim import SGD

# To get random_split to split training
# data into training and validation data
# and DataLoader to create dataloaders for train,
# valid and test data to be returned
# by our data module
from torch.utils.data import random_split, DataLoader

class DataModulePASCAL_VOC(pl.LightningDataModule):
    def __init__(self):
        super().__init__()

        # Directory to store Data
        self.download_dir = './data'

        # Defining batch size of our data
        self.batch_size = 32
        self.IMAGE_SIZE = config.IMAGE_SIZE

        # Defining transforms to be applied on the data
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])

    def prepare_data(self):
      IMAGE_SIZE = config.IMAGE_SIZE
      self.trainset_mod = YOLODataset(
          #config.DATASET + "/train.csv",
          config.DATASET + "/8examples.csv",
          transform=config.train_transforms,
          S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
          img_dir=config.IMG_DIR,
          label_dir=config.LABEL_DIR,
          anchors=config.ANCHORS,
           )
      #print(self.trainset_mod.size)


      self.testset_mod = YOLODataset(
        #config.DATASET + "/test.csv",
        config.DATASET +  "/1examples.csv",
        transform=config.test_transforms,
        S=[IMAGE_SIZE // 32, IMAGE_SIZE // 16, IMAGE_SIZE // 8],
        img_dir=config.IMG_DIR,
        label_dir=config.LABEL_DIR,
        anchors=config.ANCHORS,
       )


    def setup(self, stage=None):

          # Loading our data after applying the transforms
        self.train_data = self.trainset_mod

        self.valid_data = self.testset_mod

        self.test_data = self.testset_mod

    def train_dataloader(self):

          # Generating train_dataloader
        return DataLoader(dataset=self.train_data,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
        pin_memory=config.PIN_MEMORY,
        shuffle=True,
        drop_last=False,
    )
    def val_dataloader(self):

          # Generating val_dataloader
        return DataLoader(dataset= self.valid_data,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
        pin_memory=config.PIN_MEMORY,
        shuffle=True,
        drop_last=False,
    )

    def test_dataloader(self):
      # Generating test_dataloader
        return DataLoader(
        dataset=self.test_data,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
        pin_memory=config.PIN_MEMORY,
        shuffle=False,
        drop_last=False,
        )




In [ ]:
pascal_voc_dm = DataModulePASCAL_VOC()

In [ ]:
scaler = torch.cuda.amp.GradScaler()



In [ ]:

scaled_anchors = (
    torch.tensor(config.ANCHORS)
    * torch.tensor(config.S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
).to(config.DEVICE)

# Model

In [ ]:
class Pascal_VOC(LightningModule):
    def __init__(self):
        super().__init__()

        self.save_hyperparameters()
        self.num_classes =20

        self.model = YOLOv3(num_classes=config.NUM_CLASSES).to(config.DEVICE)

        self.loss_fn = YoloLoss()

    def forward(self, x):
        return self.model(x)


    def training_step(self, batch, batch_idx):
        #loop = tqdm(batch, leave=True)
        self.losses = []
        x, y = batch
        x = x.to(config.DEVICE)
        y0, y1, y2 = (
          y[0].to(config.DEVICE),
          y[1].to(config.DEVICE),
          y[2].to(config.DEVICE),
        )

        with torch.cuda.amp.autocast():
          out = self.model(x)
          self.loss = (
              self.loss_fn(out[0], y0, scaled_anchors[0])
              + self.loss_fn(out[1], y1, scaled_anchors[1])
              + self.loss_fn(out[2], y2, scaled_anchors[2])
          )

        self.losses.append(self.loss.item())
        self.optimizer.zero_grad()
        scaler.scale(self.loss).backward()
        scaler.step(self.optimizer)
        scaler.update()

        # update progress bar
        mean_loss = sum(self.losses) / len(self.losses)
        #loop.set_postfix(loss=mean_loss)

        self.log("train_loss", mean_loss)


    def evaluate(self, batch, stage=None):

          #loop = tqdm(batch, leave=True)
          tot_class_preds, correct_class = 0, 0
          tot_noobj, correct_noobj = 0, 0
          tot_obj, correct_obj = 0, 0

          x, y = batch
          x = x.to(config.DEVICE)
          with torch.no_grad():
              out = model(x)

          for i in range(3):
              y[i] = y[i].to(config.DEVICE)
              obj = y[i][..., 0] == 1 # in paper this is Iobj_i
              noobj = y[i][..., 0] == 0  # in paper this is Iobj_i

              correct_class += torch.sum(
                  torch.argmax(out[i][..., 5:][obj], dim=-1) == y[i][..., 5][obj]
              )
              tot_class_preds += torch.sum(obj)

              obj_preds = torch.sigmoid(out[i][..., 0]) > 0.5 #config.threshold
              correct_obj += torch.sum(obj_preds[obj] == y[i][..., 0][obj])
              tot_obj += torch.sum(obj)
              correct_noobj += torch.sum(obj_preds[noobj] == y[i][..., 0][noobj])
              tot_noobj += torch.sum(noobj)

          print(f"Class accuracy is: {(correct_class/(tot_class_preds+1e-16))*100:2f}%")
          print(f"No obj accuracy is: {(correct_noobj/(tot_noobj+1e-16))*100:2f}%")
          print(f"Obj accuracy is: {(correct_obj/(tot_obj+1e-16))*100:2f}%")
          pred_boxes, true_boxes = get_evaluation_bboxes1(
              batch,
              self.model,
              iou_threshold=config.NMS_IOU_THRESH,
              anchors=config.ANCHORS,
              threshold=config.CONF_THRESHOLD,
          )
          self.mapval = mean_average_precision(
              pred_boxes,
              true_boxes,
              iou_threshold=config.MAP_IOU_THRESH,
              box_format="midpoint",
              num_classes=config.NUM_CLASSES,
          )
          print(f"MAP: {self.mapval.item()}")

          self.log(f"{stage}_map", self.mapval, prog_bar=True)
          #     self.log(f"{stage}_class_acc", self.class_acc, prog_bar=True)
          #     self.log(f"{stage}no_obj_acc", self.no_obj_acc, prog_bar=True)
          #     self.log(f"{stage}obj_acc", self.obj_acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        self.optimizer = optim.Adam(self.model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY
        )
        self.loss_fn = YoloLoss()
        self.scaler = torch.cuda.amp.GradScaler()
        self.steps_per_epoch = 17000 // config.BATCH_SIZE
        self.scheduler_dict = {
            "scheduler": OneCycleLR(
                self.optimizer,
                0.01,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=self.steps_per_epoch,
            ),
            "interval": "step",
        }
        return {"optimizer": self.optimizer, "lr_scheduler": self.scheduler_dict}

In [ ]:
model = Pascal_VOC()

trainer = Trainer(
    max_epochs=2,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
    logger=CSVLogger(save_dir="logs/"),
    callbacks=[LearningRateMonitor(logging_interval="step"), TQDMProgressBar(refresh_rate=10)],
)

trainer.fit(model, pascal_voc_dm)
#trainer.test(model, datamodule=pascal_voc_dm)

In [ ]:

metrics = pd.read_csv(f"{trainer.logger.log_dir}/metrics.csv")
del metrics["step"]
metrics.set_index("epoch", inplace=True)
display(metrics.dropna(axis=1, how="all").head())
sn.relplot(data=metrics, kind="line")

In [ ]:
torch.save(model.state_dict(), "model.pth",_use_new_zipfile_serialization=False)

# Getting Misclassified Images

In [ ]:
from matplotlib import pyplot as plt
def get_incorrrect_predictions(model, loader, device):
    """Get all incorrect predictions

    Args:
        model (Net): Trained model
        loader (DataLoader): instance of data loader
        device (str): Which device to use cuda/cpu

    Returns:
        list: list of all incorrect predictions and their corresponding details
    """
    model.eval()
    incorrect = []
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.cross_entropy(output, target)
            pred = output.argmax(dim=1)
            for image, target, pred in zip(data, target, pred):
                if pred.eq(target.view_as(pred)).item() == False:
                    incorrect.append([image.cpu(), target.cpu(), pred.cpu()])

    return incorrect

def display_incorrect_images(mismatch, n=20 ):
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    display_images = mismatch[:n]
    index = 0
    fig = plt.figure(figsize=(10,5))
    for img in display_images:
        image = img[0].squeeze().to('cpu').numpy()
        pred = classes[img[2]]
        actual = classes[img[1]]
        ax = fig.add_subplot(2, 5, index+1)
        ax.axis('off')
        ax.set_title(f'\n Predicted Label : {pred} \n Actual Label : {actual}',fontsize=10)
        ax.imshow(np.transpose(image, (1, 2, 0)))
        index = index + 1
    plt.show()

In [ ]:
model = LitResnet(lr=.01)
model.load_state_dict(torch.load("model.pth", map_location=torch.device('cpu')), strict=False)

In [ ]:
import numpy as np
mis_class_images = get_incorrrect_predictions(model, cifar10_dm.test_dataloader(), 'cpu')
display_incorrect_images(mis_class_images, n=10)